# Nucleophilic aromatic substitution (SNAr) reaction

<div><img src="https://pubs.rsc.org/image/article/2017/re/c6re00109b/c6re00109b-s1_hi-res.gif" width="600"/></div>

This notebook aims to model a SNAr of 2,4-difluoronitrobenzene 1 with pyrrolidine 2 in ethanol to give a mixture: desired product ortho-substituted 3, para-substituted 4 and bis-adduct 5 as side products.

The process setup is shown below. Temperature, residence time and mole equivalent of pyrrolidine can be tuned for maximazing the yield.

<div><img src="https://pubs.rsc.org/image/article/2017/re/c6re00109b/c6re00109b-f1_hi-res.gif" width="600"/></div>

More details can be found at:
[C.A. Hone, N. Holmes, G.R. Akien, R.A. Bourne, F.L. Muller, Rapid multistep kinetic model generation from transient flow data, React. Chem. Eng. 2 (2017) 103–108. https://doi.org/10.1039/C6RE00109B](https://doi.org/10.1039/C6RE00109B)


### Parameter Setup

All parameters are represented with name and indicators of solid/gas, stream, reaction, and species dimensions.

> **Operation parameters**
>
> - ("Temperature", None, None, None, None) [No dimension related to temperature]
> - ("Residence_Time", None, None, None, None) [No dimension related to residence time]
> - ("Concentration", None, 0, None, 1) [Concentration of pyrrolidine]  
>  
> **Kinetics parameters**
> - ("Activation_Energy", None, 0, 0, None) [Activation energy is related to stream and reaction dimensions]
> - ("Pre-exponential_Factor", None, 0, 0, None) [Pre-exponential factor is related to stream and reaction dimensions]

### Unit Setup

Parameter setup can be found in method `var2unit`

| Parameter              | Unit   |
| ---------------------- | ------ |
| Pre-exponential_Factor |        |
| Activation_Energy      | kJ/mol |
| Temperature            | oC     |
| Concentration          | mol/L  |
| Residence_Time         | min    |

In [1]:
# import required python libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode

from cases.pyrrolidine_snar import PyrrolidineSNAr

init_notebook_mode(connected=True)

# define phenomenon dictionary for the process
phenos = {
    "Mass accumulation":    "Continuous",
    "Flow pattern":         "Tubular_Flow",
    "Mass transport":       [],
    "Mass equilibrium":     [],
}

pyrrolidine_snar = PyrrolidineSNAr(phenos)

### Model Simulation
In this section, we run simulation for the SNAr reaction with published kinetic parameters.  
Concetration landscapes can be obtained by running simulation on the mesh grid of operation parameters.

In [2]:
# list of operation parameters (dfnb concentration is fixed to 0.2)
pyrrolidine_snar.operation_params()

{('Concentration', None, 0, None, 0): 0.2,
 ('Concentration', None, 0, None, 1): None,
 ('Temperature', None, None, None, None): None,
 ('Residence_Time', None, None, None, None): None}

In [3]:
# model simulation and plot concentration profiles
# the `run` method returns the time series `t` and species concentrations `cs`
# `t` shape:  (Num. of time points, )
# `cs` shape: (Num. of species, Num. of time points, )
operation_params = {
    ("Temperature", None, None, None, None): 100,
    ("Residence_Time", None, None, None, None): 1,
    ("Concentration", None, 0, None, 1): 0.4,
}
t, cs = pyrrolidine_snar.run(operation_params)

data = {"Time (min)": [], "Concentration (mol/L)": [], "Species": []}
for i, s in enumerate(pyrrolidine_snar.species):
    for _t, _c in zip(t, cs[i]):
        data["Time (min)"].append(_t)
        data["Concentration (mol/L)"].append(_c)
        data["Species"].append(s)
df = pd.DataFrame(data)
fig = px.line(df, x="Time (min)", y="Concentration (mol/L)", color="Species")
fig.update_layout(width=800, height=500)
fig.show()

In [4]:
# Target product concentration profiles under varying temperature
temperatures = np.linspace(30, 120, 4)
data = {"Time (min)": [], "Ortho concentration (mol/L)": [], "Temperature (oC)": []}
for temperature in temperatures:
    operation_params = {
        ("Temperature", None, None, None, None): temperature,
        ("Residence_Time", None, None, None, None): 1,
        ("Concentration", None, 0, None, 1): 0.4,
    }
    t, cs = pyrrolidine_snar.run(operation_params)
    for _t, _c in zip(t, cs[pyrrolidine_snar.species.index("ortho")]):
        data["Time (min)"].append(_t)
        data["Ortho concentration (mol/L)"].append(_c)
        data["Temperature (oC)"].append(temperature)
df = pd.DataFrame(data)
fig = px.line(df, x="Time (min)", y="Ortho concentration (mol/L)", color="Temperature (oC)")
fig.update_layout(width=800, height=500)
fig.show()

In [5]:
# Target product concentration profiles under varying pyrrolidine concentration
prld_concs = np.linspace(0.1, 0.5, 5)
data = {"Time (min)": [], "Ortho concentration (mol/L)": [], "Pyrrolidine concentration (mol/L)": []}
for prld_conc in prld_concs:
    operation_params = {
        ("Temperature", None, None, None, None): 100,
        ("Residence_Time", None, None, None, None): 1,
        ("Concentration", None, 0, None, 1): prld_conc,
    }
    t, cs = pyrrolidine_snar.run(operation_params)
    for _t, _c in zip(t, cs[pyrrolidine_snar.species.index("ortho")]):
        data["Time (min)"].append(_t)
        data["Ortho concentration (mol/L)"].append(_c)
        data["Pyrrolidine concentration (mol/L)"].append(round(prld_conc, 1))
df = pd.DataFrame(data)
fig = px.line(df, x="Time (min)", y="Ortho concentration (mol/L)", color="Pyrrolidine concentration (mol/L)")
fig.update_layout(width=900, height=500)
fig.show()

In [6]:
# Target product concentration landscapes
temperatures = np.linspace(30, 120, 3)
residence_times = np.linspace(0.5, 2, 16)
prld_concs = np.linspace(0.1, 0.5, 21)
shape = (len(residence_times), len(prld_concs))

data = []
for temperature in temperatures:
    d = {
        "Temperature (oC)": temperature,
        "Residence time (min)": [], 
        "Pyrrolidine concentration (mol/L)": [], 
        "Ortho concentration (mol/L)": [], 
    }
    for residence_time in residence_times:
        for prld_conc in prld_concs:
            operation_params = {
                ("Temperature", None, None, None, None): temperature,
                ("Residence_Time", None, None, None, None): residence_time,
                ("Concentration", None, 0, None, 1): prld_conc,
            }
            t, cs = pyrrolidine_snar.run(operation_params)
            d["Residence time (min)"].append(residence_time)
            d["Pyrrolidine concentration (mol/L)"].append(prld_conc)
            d["Ortho concentration (mol/L)"].append(cs[2][-1])
    d["Residence time (min)"] = np.array(d["Residence time (min)"]).reshape(shape)
    d["Pyrrolidine concentration (mol/L)"] = np.array(d["Pyrrolidine concentration (mol/L)"]).reshape(shape)
    d["Ortho concentration (mol/L)"] = np.array(d["Ortho concentration (mol/L)"]).reshape(shape)
    data.append(d)

fig = go.Figure(
    data=[
        go.Surface(
            x=d["Residence time (min)"], 
            y=d["Pyrrolidine concentration (mol/L)"], 
            z=d["Ortho concentration (mol/L)"], 
            coloraxis="coloraxis",
        ) for d in data
    ] + [
        go.Scatter3d(
            x=[d["Residence time (min)"][0, -1]], 
            y=[d["Pyrrolidine concentration (mol/L)"][0, -1]], 
            z=[d["Ortho concentration (mol/L)"][0, -1]], 
            mode="text",
            text=[f"T = {d['Temperature (oC)']} oC"],
            textposition="bottom right",
            textfont=dict(size=12, color="red"),
            showlegend=False,
        ) for d in data
    ]
)

fig.update_layout(
    scene=dict(
        xaxis=dict(tickmode="array", tickvals=[0.5, 1, 1.5, 2], title="Residence time (min)"),
        yaxis=dict(tickmode="array", tickvals=[0.1, 0.2, 0.3, 0.4, 0.5], title="Pyrrolidine concentration (M)"),
        zaxis=dict(tickmode="array",tickvals=[0.05, 0.1, 0.15, 0.2], title="Ortho concentration (M)"),
    ),
    coloraxis=dict(colorscale="Viridis", cmin=0.06, cmax=0.18),
    width=900, height=700,
    scene_camera = dict(eye=dict(x=1.5, y=1.5, z=1.5))
)
fig.show()

### Model calibration
In this section, we run calibration for the reaction kinetics parameters for the SNAr reaction.  
Concetration landscapes can be obtained by running simulation on the mesh grid of operation parameters.

Arrhenius can be given as: $k = A \cdot exp(-\frac{E_a}{RT})$  
In the model, the Arrhenius referenced to 90 oC is used: $k = A \cdot exp(-\frac{E_a}{RT}(\frac{1}{T} - \frac{1}{363.15}))$  
In this case, the pre-exponential factor $A$ and activation energy $E_a$ are kinetics parameters to be calibrated  
For Arrhenius, $ln(k)$ is linear with $\frac{1}{T}$
<div><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Arrhenius_plot_with_break_in_y-axis_to_show_intercept.svg/500px-Arrhenius_plot_with_break_in_y-axis_to_show_intercept.svg.png" width="600"/></div>

In [7]:
# print mapping from index to name for measure parameters
print(pyrrolidine_snar.measure_ind2name())

# print mapping from name to index for operation parameters
print(pyrrolidine_snar.operation_name2ind())

{('Concentration', None, 0, None, 2): 'outlet_ortho_conc'}
{'prld_conc': ('Concentration', None, 0, None, 1), 'temp': ('Temperature', None, None, None, None), 't_r': ('Residence_Time', None, None, None, None)}


In [8]:
# generate dataset via latin hypercube sampling and add a bit of noise (±2%)
operation_param_ranges = {
    "prld_conc":    (0.1, 0.5),
    "temp":         (30, 130),
    "t_r":          (0.5, 2),
}
dataset = pyrrolidine_snar.generate_lhs_dataset(operation_param_ranges, 7)
dataset["outlet_ortho_conc"] *= (1 + (np.random.random(size=7) - 0.5) / 25)
dataset

,prld_conc,temp,t_r,outlet_ortho_conc
0,0.103261,54.052326,1.416641,0.084646
1,0.321394,109.671766,0.789706,0.171184
2,0.268190,89.729757,1.299565,0.183180
3,0.346900,60.647489,0.695006,0.167371
4,0.162236,31.124407,1.760719,0.098919
5,0.470929,77.594675,1.141354,0.175777
6,0.417511,123.336172,1.957107,0.115189


In [9]:
# calibrate kinetics parameters by minimising the mean squared error of ortho product
cal_kinetics_param_ranges = {
    ("Pre-exponential_Factor", None, 0, 0, None):   (1e-4, 1e0),
    ("Pre-exponential_Factor", None, 0, 1, None):   (1e-4, 1e0),
    ("Pre-exponential_Factor", None, 0, 2, None):   (1e-4, 1e0),
    ("Pre-exponential_Factor", None, 0, 3, None):   (1e-4, 1e0),
    ("Activation_Energy", None, 0, 0, None):        (20, 60),
    ("Activation_Energy", None, 0, 1, None):        (20, 60),
    ("Activation_Energy", None, 0, 2, None):        (20, 60),
    ("Activation_Energy", None, 0, 3, None):        (20, 60),
}
cal_kinetics_params = pyrrolidine_snar.calibrate(cal_kinetics_param_ranges, dataset)
cal_kinetics_params

{('Pre-exponential_Factor', None, 0, 0, None): 0.861259,
 ('Pre-exponential_Factor', None, 0, 1, None): 0.050457,
 ('Pre-exponential_Factor', None, 0, 2, None): 0.008416,
 ('Pre-exponential_Factor', None, 0, 3, None): 0.605027,
 ('Activation_Energy', None, 0, 0, None): 39.998215,
 ('Activation_Energy', None, 0, 1, None): 40.00273,
 ('Activation_Energy', None, 0, 2, None): 40.00225,
 ('Activation_Energy', None, 0, 3, None): 39.99995}

In [10]:
# compare ortho product concentration landscape of the calibrated model against the ground-truth at 100 oC
# Target product concentration landscapes
temperature = 100
residence_times = np.linspace(0.5, 2, 16)
prld_concs = np.linspace(0.1, 0.5, 21)
shape = (len(residence_times), len(prld_concs))

gt_data = {
    "Temperature (oC)": 100,
    "Residence time (min)": [], 
    "Pyrrolidine concentration (mol/L)": [], 
    "Ortho concentration (mol/L)": [], 
}
for residence_time in residence_times:
    for prld_conc in prld_concs:
        operation_params = {
            ("Temperature", None, None, None, None): temperature,
            ("Residence_Time", None, None, None, None): residence_time,
            ("Concentration", None, 0, None, 1): prld_conc,
        }
        t, cs = pyrrolidine_snar.run(operation_params)
        gt_data["Residence time (min)"].append(residence_time)
        gt_data["Pyrrolidine concentration (mol/L)"].append(prld_conc)
        gt_data["Ortho concentration (mol/L)"].append(cs[2][-1])
gt_data["Residence time (min)"] = np.array(gt_data["Residence time (min)"]).reshape(shape)
gt_data["Pyrrolidine concentration (mol/L)"] = np.array(gt_data["Pyrrolidine concentration (mol/L)"]).reshape(shape)
gt_data["Ortho concentration (mol/L)"] = np.array(gt_data["Ortho concentration (mol/L)"]).reshape(shape)

cal_data = {
    "Temperature (oC)": 100,
    "Residence time (min)": [], 
    "Pyrrolidine concentration (mol/L)": [], 
    "Ortho concentration (mol/L)": [], 
}
for residence_time in residence_times:
    for prld_conc in prld_concs:
        operation_params = {
            ("Temperature", None, None, None, None): temperature,
            ("Residence_Time", None, None, None, None): residence_time,
            ("Concentration", None, 0, None, 1): prld_conc,
        }
        t, cs = pyrrolidine_snar.run(operation_params, cal_kinetics_params)
        cal_data["Residence time (min)"].append(residence_time)
        cal_data["Pyrrolidine concentration (mol/L)"].append(prld_conc)
        cal_data["Ortho concentration (mol/L)"].append(cs[2][-1])
cal_data["Residence time (min)"] = np.array(cal_data["Residence time (min)"]).reshape(shape)
cal_data["Pyrrolidine concentration (mol/L)"] = np.array(cal_data["Pyrrolidine concentration (mol/L)"]).reshape(shape)
cal_data["Ortho concentration (mol/L)"] = np.array(cal_data["Ortho concentration (mol/L)"]).reshape(shape)

fig = go.Figure()
fig.add_trace(
    go.Surface(
        x=gt_data["Residence time (min)"], 
        y=gt_data["Pyrrolidine concentration (mol/L)"], 
        z=gt_data["Ortho concentration (mol/L)"], 
        colorscale='Viridis',
        colorbar=dict(title='Ground-truth', len=0.8, x=1.05),
        cmin=0.1,
        cmax=0.185,
        name='Ground-truth',
        showscale=True,
    )
)
fig.add_trace(
    go.Surface(
        x=cal_data["Residence time (min)"], 
        y=cal_data["Pyrrolidine concentration (mol/L)"], 
        z=cal_data["Ortho concentration (mol/L)"], 
        colorscale='Thermal',
        colorbar=dict(title='Calibrated model', len=0.8, x=1.25),
        cmin=0.1,
        cmax=0.185,
        name='Calibrated model',
        showscale=True
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(tickmode="array", tickvals=[0.5, 1, 1.5, 2], title="Residence time (min)"),
        yaxis=dict(tickmode="array", tickvals=[0.1, 0.2, 0.3, 0.4, 0.5], title="Pyrrolidine concentration (M)"),
        zaxis=dict(tickmode="array",tickvals=[0.05, 0.1, 0.15, 0.2], title="Ortho concentration (M)"),
    ),
    # coloraxis=dict(colorscale="Viridis", cmin=0.06, cmax=0.18),
    width=900, height=700,
    scene_camera = dict(eye=dict(x=1.5, y=1.5, z=1.5))
)
fig.show()